<a href="https://colab.research.google.com/github/Lausti98/bsc-recsys/blob/main/bsc_steam_initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install recpack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 20.1 MB/s eta 0:00:00
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2


In [15]:
# import required module
from pathlib import Path
import pandas as pd
import re
from sklearn import preprocessing
fpath = 'steam-200k.csv'

df = pd.read_csv(fpath, names=['user_id', 'game_id', 'interaction', 'rating', 'n'])#, error_bad_lines=False, skiprows=190000)
df = df[df['interaction'].astype('string') == 'play'] # filter out purchase interactions
df = df.drop(columns=['interaction', 'n'])
le = preprocessing.LabelEncoder()
df['user_id'] = le.fit_transform(df['user_id'])
df['game_id'] = le.fit_transform(df['game_id'])
print(df.dtypes)
df


user_id      int64
game_id      int64
rating     float64
dtype: object


,user_id,game_id,rating
1,5088,3067,273.0
3,5088,1162,87.0
5,5088,2813,14.9
7,5088,1163,12.1
9,5088,1733,8.9
...,...,...,...
199991,4056,1155,2.4
199993,4056,1833,2.2
199995,4056,3220,1.5
199997,4056,1375,1.5


In [17]:
from recpack.preprocessing.preprocessors import DataFramePreprocessor
from recpack.preprocessing.filters import MinItemsPerUser, MinUsersPerItem

proc = DataFramePreprocessor(item_ix='game_id', user_ix='user_id')#, timestamp_ix='ts')
proc.add_filter(MinUsersPerItem(5, item_ix='game_id', user_ix='user_id'))
proc.add_filter(MinItemsPerUser(5, item_ix='game_id', user_ix='user_id'))

# Assuming you have loaded a dataframe called df
interaction_matrix = proc.process(df)

  0%|          | 0/54083 [00:00<?, ?it/s]

  0%|          | 0/54083 [00:00<?, ?it/s]

In [18]:
from recpack.scenarios import WeakGeneralization

# Split data into train, test and validation set. Random split not accounting for time data.
scenario = WeakGeneralization(0.75, validation=True)
scenario.split(interaction_matrix)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [19]:
from recpack.pipelines import PipelineBuilder

builder = PipelineBuilder()

builder.set_data_from_scenario(scenario)

In [20]:
builder.add_algorithm('Popularity') # No real parameters to optimise

In [21]:
# ITEM BASED Nearest Neighbors Recommendation Recommender Systems: An Introduction 2.2
builder.add_algorithm('ItemKNN', grid={
    'K': [100, 200, 500],
    'similarity': ['cosine'],# 'conditional_probability'],
})

#builder.add_algorithm('SLIM')

#builder.add_algorithm('KUNN', grid={
#    'Ku': [100, 200, 500],
#    'Ki': [100, 200, 500],
#})

/usr/local/lib/python3.8/dist-packages/recpack/pipelines/pipeline_builder.py:145: UserWarning: Grid parameter for add_algorithm function will be deprecated in favour of optimisation_info.
  warnings.warn(


In [22]:
builder.set_optimisation_metric('NDCGK', K=10)
builder.add_metric('NDCGK', K=[10, 20, 50])
builder.add_metric('CoverageK', K=[10, 20])
builder.add_metric('RecallK', K=[10, 20, 50])
builder.add_metric('PrecisionK', K=[10, 20, 50])

In [23]:
pipeline = builder.build()
pipeline.run()
pipeline.get_metrics()

  0%|          | 0/2 [00:00<?, ?it/s]

2023-03-03 10:43:44,587 - base - recpack - INFO - Fitting Popularity complete - Took 0.0233s


INFO:recpack:Fitting Popularity complete - Took 0.0233s
/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


2023-03-03 10:43:46,431 - base - recpack - INFO - Fitting ItemKNN complete - Took 0.235s


/usr/local/lib/python3.8/dist-packages/recpack/algorithms/base.py:279: UserWarning: ItemKNN missing similar items for 8 items.
  warnings.warn(f"{self.name} missing similar items for {missing} items.")
INFO:recpack:Fitting ItemKNN complete - Took 0.235s


...

2023-03-03 10:43:48,986 - base - recpack - INFO - Fitting ItemKNN complete - Took 1.16s


INFO:recpack:Fitting ItemKNN complete - Took 1.16s


2023-03-03 10:43:50,929 - base - recpack - INFO - Fitting ItemKNN complete - Took 1.05s


INFO:recpack:Fitting ItemKNN complete - Took 1.05s


,NDCGK_10,NDCGK_20,NDCGK_50,CoverageK_10,CoverageK_20,RecallK_10,RecallK_20,RecallK_50,PrecisionK_10,PrecisionK_20,PrecisionK_50
Popularity(K=200),0.174141,0.195392,0.225902,0.018939,0.031566,0.198563,0.279364,0.386196,0.074405,0.052797,0.032601
"ItemKNN(K=500,normalize_X=False,normalize_sim=False,pop_discount=None,similarity=cosine)",0.278475,0.299650,0.333545,0.354798,0.518308,0.320859,0.402615,0.521089,0.107056,0.072505,0.042372
